In [ ]:
import os
import pandas as pd
import sys
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Caminhos dos seus módulos customizados
sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases
from parametros import Parametros

diretorio_base = r"C:\Scripts\relatórios"

In [ ]:
# %%
pastas_dias = []

# Coleta as pastas diretamente na raiz de relatórios (pulando a pasta mês)
for dia in next(os.walk(r"C:\\Scripts\\relatórios"))[1]:
    try:
        datetime.strptime(dia, "%d-%m-%Y")
        pastas_dias.append({"dia": dia})
    except ValueError:
        continue

# Ordena igual ao original
pastas_dias.sort(key = lambda date: datetime.strptime(date["dia"], "%d-%m-%Y"))

# Lista para armazenar os DataFrames de cada dia
lista_entrada_saida = []

for presente in pastas_dias:
    dia_str = presente['dia']
    diretorio_pastas_presente = os.path.join(diretorio_base, dia_str)
    
    try:
        subpastas = os.listdir(diretorio_pastas_presente)
        if not subpastas:
            continue
            
        base_hoje = subpastas[0]
        caminho_final = os.path.join(diretorio_pastas_presente, base_hoje, "base_btg.xlsx")
        
        # Tenta ler o arquivo
        try:
            df = pd.read_excel(caminho_final, header=2)
            if 'Conta' not in df.columns:
                df = pd.read_excel(caminho_final)
        except:
            df = pd.read_excel(caminho_final)

        # Padroniza colunas (Remove " (R$)" e espaços)
        df.columns = [str(c).replace(" (R$)", "").strip() for c in df.columns]

        # Mostra o dia e as colunas originais encontradas antes de renomear
        print(f"--- Processando dia: {dia_str} ---")
        print(f"Colunas encontradas: {list(df.columns)}")

        # Tratamento da coluna de abertura
        mapeamento = {'Data de Abertura da Conta': 'Data de Abertura', 'Data Vínculo Assessor': 'Data Vínculo'}
        df.rename(columns=mapeamento, inplace=True)

        # Seleção das colunas
        colunas_alvo = ['Conta', 'Assessor', 'Data de Abertura', 'Data Vínculo', 'Faixa Cliente', 'PL Total']
        colunas_existentes = [c for c in colunas_alvo if c in df.columns]
        
        # DEBUG: Confirma o que foi selecionado após o tratamento
        print(f"Colunas selecionadas: {colunas_existentes}\n")

        df_final = df[colunas_existentes].copy()
        
        if 'Conta' in df_final.columns:
            df_final['Conta'] = df_final['Conta'].astype(str)

        lista_entrada_saida.append(df_final)

    except Exception as e:
        print(f"!!! ERRO no dia {dia_str}: {e}\n")

historico_abertura = pd.concat(lista_entrada_saida, ignore_index=True)

In [ ]:
lista_entrada_saida = []
diretorio_base = r"C:\Scripts\relatórios"

pastas_processadas = 0
datas_lidas = []

print("Iniciando processamento...\n")

for presente in pastas_dias:
    dia_str = presente['dia']
    caminho_dia = os.path.join(diretorio_base, dia_str)
    
    try:
        subpastas = os.listdir(caminho_dia)
        if not subpastas:
            continue
            
        pasta_hora = subpastas[0]
        caminho_arquivo = os.path.join(caminho_dia, pasta_hora, "base_btg.xlsx")

        # Leitura do Excel
        try:
            df = pd.read_excel(caminho_arquivo, header=2)
            if 'Conta' not in df.columns:
                df = pd.read_excel(caminho_arquivo)
        except:
            df = pd.read_excel(caminho_arquivo)

        # Padronização e Mapeamento
        df.columns = [str(c).replace(" (R$)", "").strip() for c in df.columns]
        
        mapeamento = {
            'Data de Abertura da Conta': 'Data de Abertura',
            'Data de Abertura do Assessor': 'Data de Abertura',
            'Data Vínculo Assessor': 'Data Vínculo'
        }
        df.rename(columns=mapeamento, inplace=True)

        colunas_alvo = ['Conta', 'Assessor', 'Data de Abertura', 'Data Vínculo', 'Faixa Cliente', 'PL Total']
        colunas_final = [c for c in colunas_alvo if c in df.columns]
        
        base_atual = df[colunas_final].copy()
        if 'Conta' in base_atual.columns:
            base_atual['Conta'] = base_atual['Conta'].astype(str)

        lista_entrada_saida.append(base_atual)
        
        # Atualiza contadores para o Debug
        pastas_processadas += 1
        datas_lidas.append(datetime.strptime(dia_str, "%d-%m-%Y"))

        # Log individual simplificado para não poluir o console
        print(f"[OK] {dia_str} | Colunas: {len(colunas_final)}")

    except Exception as e:
        print(f"[ERRO] Dia {dia_str}: {e}")

# --- RESUMO DO DEBUG ---
print("\n" + "="*30)
print("      RESUMO DO PROCESSO")
print("="*30)
if datas_lidas:
    print(f"Total de pastas lidas: {pastas_processadas}")
    print(f"Data inicial: {min(datas_lidas).strftime('%d/%m/%Y')}")
    print(f"Data final:   {max(datas_lidas).strftime('%d/%m/%Y')}")
    print(f"Última pasta lida: {dia_str}")
else:
    print("Nenhuma pasta foi processada.")
print("="*30 + "\n")

if lista_entrada_saida:
    historico_abertura = pd.concat(lista_entrada_saida, ignore_index=True)

In [ ]:
historico_abertura = pd.concat(lista_entrada_saida)

In [ ]:
historico_abertura = \
    historico_abertura[
        ~((historico_abertura['Assessor'] == 'Gabriel Rodrigues') &
        (historico_abertura['Data Vínculo'] >= "2024-02-01") &
        (historico_abertura['Data Vínculo'] <= "2024-03-01"))
    ]
    
historico_abertura = \
    historico_abertura[
        ~((historico_abertura['Assessor'] == 'Vinicius Servino Vargas') &
        (historico_abertura['Data Vínculo'] == "2024-02-08"))
    ]

In [ ]:
conn = Connect.connect_techdb()
entradas_saidas_consolidado = Connect.import_table(conn, 'Entradas_e_saidas_consolidado')
conn.close()

In [ ]:
entradas_saidas_consolidado

In [ ]:
sairam_entraram = historico_abertura.merge(entradas_saidas_consolidado[entradas_saidas_consolidado['Situação'] == 'Saiu'][['Conta', 'Mês de entrada/saída']], on='Conta', how='left')
sairam_entraram = \
    sairam_entraram[
        (sairam_entraram['Mês de entrada/saída'] < sairam_entraram['Data Vínculo'])
    ]
sairam_entraram.drop("Mês de entrada/saída", axis=1, inplace=True)

In [ ]:
historico_abertura = historico_abertura[~historico_abertura['Conta'].isin(sairam_entraram['Conta'])]
historico_abertura = pd.concat([historico_abertura, sairam_entraram], axis=0)

In [ ]:
historico_abertura.drop_duplicates(subset=["Conta"], keep='first', inplace=True)
historico_abertura.rename(columns={"Faixa Cliente":"Faixa Cliente Abertura", "PL Total":"PL Abertura"}, inplace=True)

In [ ]:
primeiro_vinculo = historico_abertura[['Assessor', 'Conta', 'Data Vínculo', 'Faixa Cliente Abertura', 'PL Abertura']]
#primeiro_vinculo.rename(columns={"Data Vínculo":"Primeiro Vínculo"}, inplace=True)

In [ ]:
conexao = Connect.connect_techdb()
basebtg = Connect.import_table(conexao, "base_btg")
conexao.close()

# 1. Padroniza os nomes das colunas (remove espaços extras que o banco pode trazer)
basebtg.columns = basebtg.columns.str.strip()

# 2. Mapeia todas as versões possíveis da coluna de abertura
# Se o banco trouxer o nome novo, ele converte corretamente
mapeamento_abertura = {
    'Data de Abertura do Assessor': 'Data de Abertura',
    'Data de Abertura da Conta': 'Data de Abertura'
}
basebtg.rename(columns=mapeamento_abertura, inplace=True)

# 3. Faz o merge apenas com as colunas que agora temos certeza que existem
try:
    base_vinculos = primeiro_vinculo.merge(
        basebtg[['Conta', 'Data de Abertura']], 
        on='Conta', 
        how='left'
    )
    base_vinculos = base_vinculos.dropna(subset="Data Vínculo")
    print("Merge realizado com sucesso usando a coluna: Data de Abertura" )
except KeyError:
    # Caso o banco tenha um nome totalmente diferente, esse debug te avisará qual é
    print(f"Erro: Não achei a coluna de abertura. Colunas no banco são: {basebtg.columns.tolist()}")

In [ ]:
base_vinculos = primeiro_vinculo.merge(basebtg[['Conta', 'Data de Abertura']], on='Conta', how='left')
base_vinculos = base_vinculos.dropna(subset="Data Vínculo")

In [ ]:
base_filtrada = base_vinculos.copy()

In [ ]:
origem = pd.read_excel(r'C:\Scripts\backups_atria\historico_relatorios\historico_nnm\NNM Válido Potenza - Parcial 07.02.2023.xlsx')
origem = origem[['Conta', 'Origem da Conta']]
origem['Conta'] = origem['Conta'].astype(str)
origem.drop_duplicates(subset="Conta", inplace=True)

In [ ]:
base_filtrada = base_filtrada.merge(origem, on='Conta', how='left')

In [ ]:
base_filtrada = base_filtrada[[
    'Conta', 'Assessor','Data de Abertura', 'Data Vínculo', 
    'Faixa Cliente Abertura', 'Origem da Conta', 'PL Abertura'
    ]]

In [ ]:
base_filtrada['Tipo'] = ''

In [ ]:
base_filtrada.loc[
    (base_filtrada['Data de Abertura'] == base_filtrada['Data Vínculo']),
    "Tipo"
] = "Abertura"

In [ ]:
base_filtrada.loc[
    (base_filtrada['Data Vínculo'] != base_filtrada['Data de Abertura']),
    "Tipo"
] = "Migração"

In [ ]:
base_filtrada = base_filtrada[~base_filtrada['Data de Abertura'].isna()]

In [ ]:
base_filtrada['Mes Abertura'] = [data.strftime("%Y-%m") for data in base_filtrada['Data de Abertura']]
base_filtrada['Mes Vinculo'] = [data.strftime("%Y-%m") for data in base_filtrada['Data Vínculo']]
base_filtrada['PL Abertura'].fillna(0, inplace=True)

In [ ]:
base_filtrada = base_filtrada[
                    (base_filtrada['Tipo'] != "Migração Interna") &
                    (base_filtrada['Tipo'] != "")
                    ]

In [ ]:
base_filtrada = pd.merge(base_filtrada, basebtg[['Conta', 'PL Total']], on='Conta', how='left')
base_filtrada['PL Total'].fillna(0, inplace=True)
base_filtrada.rename(columns={"PL Total":"PL Atual"}, inplace=True)

In [ ]:
conexao = Connect.connect_techdb()
pl_historico = Connect.import_table(conexao, "PL Base")
conexao.close()

In [ ]:
contas_pl = base_filtrada['Conta']

for conta in contas_pl:
    try:
        if len(pl_historico[(pl_historico['CONTA'] == conta)].sort_values("Mês")) > 1:
            pl_mes_abertura = pl_historico[(pl_historico['CONTA'] == conta)].sort_values("Mês").iloc[1,:]['PL']
            base_filtrada.loc[(base_filtrada['Conta'] == conta), 'PL Abertura'] = pl_mes_abertura
        else:
            pl_mes_abertura = pl_historico[(pl_historico['CONTA'] == conta)].sort_values("Mês")['PL'].values[0]
            base_filtrada.loc[(base_filtrada['Conta'] == conta), 'PL Abertura'] = pl_mes_abertura
    except:
        pass

In [ ]:
base_filtrada.loc[base_filtrada['PL Abertura'] < 300000, "Faixa Cliente Abertura"] = "< 300k"
base_filtrada.loc[(base_filtrada['PL Abertura'] >= 300000) & (base_filtrada['PL Abertura'] < 1000000), "Faixa Cliente Abertura"] = "> 300k e < 1mm"
base_filtrada.loc[(base_filtrada['PL Abertura'] >= 1000000) & (base_filtrada['PL Abertura'] < 5000000), "Faixa Cliente Abertura"] = "> 1mm e < 5mm"
base_filtrada.loc[base_filtrada['PL Abertura'] > 5000000, "Faixa Cliente Abertura"] = "> 5mm"

In [ ]:
base_filtrada.loc[base_filtrada['PL Atual'] < 300000, "Faixa Cliente Atual"] = "< 300k"
base_filtrada.loc[(base_filtrada['PL Atual'] >= 300000) & (base_filtrada['PL Atual'] < 1000000), "Faixa Cliente Atual"] = "> 300k e < 1mm"
base_filtrada.loc[(base_filtrada['PL Atual'] >= 1000000) & (base_filtrada['PL Atual'] < 5000000), "Faixa Cliente Atual"] = "> 1mm e < 5mm"
base_filtrada.loc[base_filtrada['PL Atual'] > 5000000, "Faixa Cliente Atual"] = "> 5mm"

In [ ]:
base_filtrada.loc[base_filtrada['PL Atual'] < 1000000, "Faixa Cliente Performance"] = "< 1MM"
base_filtrada.loc[base_filtrada['PL Atual'] >= 1000000, "Faixa Cliente Performance"] = "> 1MM"

In [ ]:
base_filtrada.loc[base_filtrada['Assessor'] == "Rodrigo de Mello D?Elia", "Assessor"] = "Rodrigo de Mello D’Elia"
base_filtrada.loc[base_filtrada['Assessor'] == "Rodrigo de Mello DElia", "Assessor"] = "Rodrigo de Mello D’Elia"

In [ ]:
base_filtrada['Assessor'] = [nome.upper() for nome in base_filtrada['Assessor']]

In [ ]:
tempo_potenza = base_filtrada[['Conta', 'Data Vínculo', 'Assessor']]

In [ ]:
tempo_potenza

In [ ]:
conexao = Connect.connect_techdb()
tempo_potenza.to_sql("primeiro_vinculo", 
                     con=conexao, 
                     index=False, 
                     if_exists='replace',
                     schema='principal')
conexao.close()

In [ ]:
base_filtrada = base_filtrada[
        (base_filtrada['Data Vínculo'] >= "2022-01-01") |
        (base_filtrada['Data de Abertura'] >= "2022-01-01")
    ]

In [ ]:
nomes_clientes = pd.read_excel(r"C:\\Scripts\\nomes_clientes\\Nomes_Clientes2.xlsx")

In [ ]:
base_filtrada = base_filtrada.merge(nomes_clientes, on='Conta', how='left')

In [ ]:
base_filtrada

In [ ]:
base_filtrada.loc[base_filtrada['Conta'] == '590732', "Assessor"] = "JOSE AUGUSTO ALVES DE PAULA FILHO"
base_filtrada.loc[base_filtrada['Conta'] == '299305', "Assessor"] = "JOSE AUGUSTO ALVES DE PAULA FILHO"

In [ ]:
# %%
# Ordena pela Data Vínculo do mais recente para o mais antigo
base_recente = base_filtrada.sort_values(by='Data Vínculo', ascending=False)

print(f"--- Top 10 registros com as datas de vínculo mais recentes ---")
print(f"Total de linhas na base: {len(base_recente)}")

# Mostra as colunas principais para facilitar a conferência
display(base_recente[['Conta', 'Assessor', 'Data de Abertura', 'Data Vínculo', 'Tipo', 'PL Atual']].head(10))

# %%
# DEBUG: Verificando se existem datas nulas que podem estar fugindo da ordenação
nulos = base_recente['Data Vínculo'].isna().sum()
if nulos > 0:
    print(f"Atenção: Existem {nulos} linhas com Data Vínculo vazia.")

In [ ]:
conexao = Connect.connect_techdb()
base_filtrada.to_sql("abertura_de_conta_base", 
                     con=conexao, 
                     index=False, 
                     if_exists='replace',
                     schema='principal')
conexao.close()

In [ ]:
abertura = base_filtrada[base_filtrada['Tipo'] == "Abertura"]
abertura = abertura.groupby(["Assessor", "Mes Abertura", "Faixa Cliente Abertura", "Faixa Cliente Atual", "Faixa Cliente Performance"])['Data de Abertura'].count().reset_index()

In [ ]:
vinculo = base_filtrada[base_filtrada['Tipo'] == "Migração"]
vinculo = vinculo.groupby(["Assessor", "Mes Vinculo", "Faixa Cliente Abertura", "Faixa Cliente Atual", "Faixa Cliente Performance"])['Data Vínculo'].count().reset_index()

In [ ]:
abertura.rename(columns={"Mes Abertura":"Mes", "Data de Abertura":"Contas Abertas"}, inplace=True)
vinculo.rename(columns={"Mes Vinculo":"Mes", "Data Vínculo":"Contas Vinculadas"}, inplace=True)

In [ ]:
df = abertura.merge(vinculo, on=['Assessor', 'Mes', 'Faixa Cliente Abertura', 'Faixa Cliente Atual', 'Faixa Cliente Performance'], how='outer')

In [ ]:
# 1. Encontra a data mínima e máxima dentro da sua base tratada
# Usamos 'pd.to_datetime' para garantir que não dê erro de tipo
data_min = min(pd.to_datetime(base_filtrada['Data de Abertura']).min(), pd.to_datetime(base_filtrada['Data Vínculo']).min())
data_max = max(pd.to_datetime(base_filtrada['Data de Abertura']).max(), pd.to_datetime(base_filtrada['Data Vínculo']).max())

# 2. Cria a lista de datas mensais (frequência 'MS' = Month Start)
datas = pd.date_range(start=data_min, end=data_max, freq='MS')

print(f"Gerando calendário de {data_min.strftime('%m/%Y')} até {data_max.strftime('%m/%Y')}")

# --- SEU CÓDIGO ORIGINAL CONTINUA AQUI ---

df_datas_list = []

# Pegamos os valores únicos de cada coluna para criar a matriz completa
lista_assessores = df['Assessor'].unique() if 'Assessor' in df.columns else []
lista_faixas_abertura = df['Faixa Cliente Abertura'].unique() if 'Faixa Cliente Abertura' in df.columns else []
lista_faixas_atual = df['Faixa Cliente Atual'].unique() if 'Faixa Cliente Atual' in df.columns else []

# Agora o 'datas' existe e o loop vai funcionar
for data in datas:
    mes_str = data.strftime("%Y-%m")
    for assessor in lista_assessores:
        for faixa_abertura in lista_faixas_abertura:
            for faixa_atual in lista_faixas_atual:
                
                df_datas_list.append({
                    "Mes": mes_str,
                    "Assessor": assessor,
                    "Faixa Cliente Abertura": faixa_abertura,
                    "Faixa Cliente Atual": faixa_atual
                })

df_datas = pd.DataFrame(df_datas_list)

print(f"✅ Sucesso! Geradas {len(df_datas)} linhas de base para o histórico.")

In [ ]:
# %%
df_datas_list = []

# Pegamos os valores únicos de cada coluna para criar a matriz completa
# Usamos .unique() e tratamos caso a coluna não exista para o script não parar
lista_assessores = df['Assessor'].unique() if 'Assessor' in df.columns else []
lista_faixas_abertura = df['Faixa Cliente Abertura'].unique() if 'Faixa Cliente Abertura' in df.columns else []
lista_faixas_atual = df['Faixa Cliente Atual'].unique() if 'Faixa Cliente Atual' in df.columns else []

for data in datas:
    mes_str = data.strftime("%Y-%m")
    for assessor in lista_assessores:
        for faixa_abertura in lista_faixas_abertura:
            for faixa_atual in lista_faixas_atual:
                # Criamos um dicionário simples para cada linha
                df_datas_list.append({
                    "Mes": mes_str,
                    "Assessor": assessor,
                    "Faixa Cliente Abertura": faixa_abertura,
                    "Faixa Cliente Atual": faixa_atual
                })

# Transforma a lista de dicionários em DataFrame de uma vez só
# Isso substitui o pd.concat(dfs) e já cria a coluna 'Mes'
df_datas = pd.DataFrame(df_datas_list)

print(f"✅ Sucesso! Geradas {len(df_datas)} linhas de base para o histórico.")

In [ ]:

colunas_necessarias = ["Assessor", "Mes", "Faixa Cliente Abertura", "Faixa Cliente Atual"]
colunas_atuais = df.columns.tolist()

missing = [c for c in colunas_necessarias if c not in colunas_atuais]

if not missing:
    print("Todas as colunas necessárias estão presentes!")
else:
    print(f"Erro: Faltam as seguintes colunas: {missing}")
    print(f"Colunas disponíveis: {colunas_atuais}")

In [ ]:
# %%
# Realiza o merge para trazer as contagens de contas
df_final = pd.merge(
    df_datas, 
    df[['Mes', 'Assessor', 'Faixa Cliente Abertura', 'Faixa Cliente Atual', 'Contas Abertas', 'Contas Vinculadas']], 
    on=['Mes', 'Assessor', 'Faixa Cliente Abertura', 'Faixa Cliente Atual'], 
    how='left'
)

# Preenche os campos vazios com 0
df_final.fillna(0, inplace=True)

In [ ]:
conexao = Connect.connect_techdb()
times = Connect.import_table(conexao, "times_nova_empresa")
conexao.close()

In [ ]:
df_datas = df_datas[df_datas['Assessor'].isin(times['Assessor'])]

In [ ]:
df = pd.merge(df_datas, df, on=['Mes', 'Faixa Cliente Abertura', 'Faixa Cliente Atual', 'Assessor'], how='outer')

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
conexao = Connect.connect_techdb()
df.to_sql("abertura_de_conta", 
          con=conexao, 
          index=False, 
          if_exists='replace',
          schema='principal')
conexao.close()

In [ ]:
contas_abertas = \
    base_filtrada[[
        'Data de Abertura', 'Data Vínculo', 'Conta', 'Nome',
        'Assessor', 'Tipo', 'PL Abertura', 'PL Atual'
    ]]

In [ ]:
primeiro_dia_semana = (datetime.today() - timedelta(days = datetime.today().weekday())).strftime("%Y-%m-%d")

In [ ]:
contas_abertas = contas_abertas[contas_abertas['Data Vínculo'] >= primeiro_dia_semana]

In [ ]:
conexao =Connect.connect_techdb()
infos_clientes = Connect.import_table(conexao, "infos_clientes")
conexao.close()
infos_clientes.rename(columns={"Tipo":"Tipo de conta"}, inplace=True)

In [ ]:
contas_abertas = contas_abertas.merge(infos_clientes[['Conta', 'Tipo de conta', 'Profissão / Setor', 'IDADE']], on='Conta', how='left')

In [ ]:
contas_abertas.to_excel(r"C:\Scripts\backups_atria\diarios\arquivos_banco\scripts\relatorios\entradas_e_saidas\contas_novas_semana.xlsx", header=True, index=False)